In [1]:
# Packages

import gc
import polars as pl
import numpy as np 
import pandas as pd
import json
import glob
import matplotlib.pyplot as plt
import xgboost as xgb
import mlflow
import mlflow.xgboost
import warnings
from itertools import product
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
# Loading and concatenating the whole dataset

class LoadData:
    
    def __init__(self, file_paths):
        self.file_paths = file_paths
        
    def load_and_concat(self):

        partitioned_data = [pl.read_parquet(file_path) for file_path in self.file_paths]
        df = pl.concat(partitioned_data, rechunk=False)
        
        return df
    
# Specify file paths
file_paths = sorted(glob.glob('Data/train.parquet/*/*.parquet'))

# Initialize the loader and load data as a lazy frame
loader = LoadData(file_paths)
df_full = loader.load_and_concat()

In [3]:
# Import necessary libraries
warnings.filterwarnings('ignore')

# Set up MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Jane Street Forecasting Custom R2")

# Ensure df_full is sorted by 'date_id' and 'time_id'
df_full = df_full.sort(['date_id', 'time_id'])

# Define hyperparameter distributions for random search
from scipy.stats import uniform, randint

param_distributions = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.19),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 0.5),
    'min_child_weight': randint(1, 6),
    'reg_alpha': uniform(0.0, 1.0),
    'reg_lambda': uniform(0.5, 4.5)
}

# Number of hyperparameter combinations to try
n_iter = 20

# Initialize random number generator
rng = np.random.default_rng(seed=42)

# Generate random hyperparameter combinations
hyperparameter_list = []
for _ in range(n_iter):
    params = {
        'n_estimators': rng.integers(50, 500),
        'max_depth': rng.integers(3, 10),
        'learning_rate': rng.uniform(0.01, 0.2),
        'subsample': rng.uniform(0.6, 1.0),
        'colsample_bytree': rng.uniform(0.6, 1.0),
        'gamma': rng.uniform(0, 0.5),
        'min_child_weight': rng.integers(1, 6),
        'reg_alpha': rng.uniform(0.0, 1.0),
        'reg_lambda': rng.uniform(0.5, 5.0)
    }
    hyperparameter_list.append(params)


# Exclude non-informative features and the target
excluded_features = [col for col in df_full.columns if col.startswith('responder_')] + ['weight']

# Define feature columns (excluding the target and unwanted features)
feature_cols = [col for col in df_full.columns if col not in excluded_features]

# Define custom cross-validation folds
n_splits = 8
data_length = len(df_full)

# Split the data into n_splits folds
fold_sizes = np.full(n_splits, data_length // n_splits)
fold_sizes[:data_length % n_splits] += 1  

indices = np.arange(data_length)
current = 0
fold_indices_list = []
for fold_size in fold_sizes:
    start, stop = current, current + fold_size
    fold_indices_list.append(indices[start:stop])
    current = stop

# Create custom fold indices
validation_fraction = 0.2

custom_fold_indices = []

for i in range(1, n_splits):
    # Get indices for folds i - 1 and i
    fold_i_minus1_indices = fold_indices_list[i - 1]
    fold_i_indices = fold_indices_list[i]

    # Split fold_i_indices into training and validation sets
    val_size = int(len(fold_i_indices) * validation_fraction)
    if val_size == 0:
        val_size = 1 

    train_fold_i_indices = fold_i_indices[:-val_size]
    val_fold_i_indices = fold_i_indices[-val_size:]

    # Combine training indices
    train_indices = np.concatenate([fold_i_minus1_indices, train_fold_i_indices])

    # Validation indices
    val_indices = val_fold_i_indices

    custom_fold_indices.append((train_indices, val_indices))

# Define the weighted R² metric for evaluation
def weighted_r2_score(y_true, y_pred, sample_weight):
    numerator = np.sum(sample_weight * (y_true - y_pred) ** 2)
    denominator = np.sum(sample_weight * y_true ** 2)
    return 1 - (numerator / denominator)

# Define the custom evaluation metric for XGBoost
def weighted_r2_eval(preds, dtrain):
    y_true = dtrain.get_label()
    w = dtrain.get_weight()
    numerator = np.sum(w * (y_true - preds) ** 2)
    denominator = np.sum(w * y_true ** 2)
    r2 = 1 - (numerator / denominator)
    return 'weighted_r2', r2

def train_evaluate_model(df_full, feature_cols, params, custom_fold_indices):                  

    fold_weighted_r2_list = []

    for fold, (train_indices, val_indices) in enumerate(custom_fold_indices):
        print(f"\nProcessing Fold {fold + 1}")
        try:
            # Select training and validation data from Polars DataFrame
            df_train = df_full[train_indices]
            df_val = df_full[val_indices]

            # Convert Polars DataFrames to Pandas DataFrames for XGBoost
            df_train_pd = df_train.to_pandas()
            df_val_pd = df_val.to_pandas()

            # Extract features, target, and weights
            X_train = df_train_pd[feature_cols].astype(np.float32)
            y_train = df_train_pd['responder_6'].astype(np.float32)
            w_train = df_train_pd['weight'].astype(np.float32)

            X_val = df_val_pd[feature_cols].astype(np.float32)
            y_val = df_val_pd['responder_6'].astype(np.float32)
            w_val = df_val_pd['weight'].astype(np.float32)

            # Convert to XGBoost DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train, weight=w_train)
            dval = xgb.DMatrix(X_val, label=y_val, weight=w_val)

            # Set up model parameters
            model_params = params.copy()
            model_params.update({
                'objective': 'reg:squarederror',
                'tree_method': 'hist',     
                'eval_metric': 'rmse',
                'n_jobs': -1,              
                'verbosity': 0,
                'seed': 42                 
            })

            # Specify evaluation set
            evals = [(dval, 'validation')]

            # Train the model with early stopping and custom metric
            model = xgb.train(
                model_params,
                dtrain,
                num_boost_round=1000,
                evals=evals,
                early_stopping_rounds=10,
                feval=weighted_r2_eval,
                maximize=True,  # Because higher R² is better
                verbose_eval=False
            )

            # Predict and evaluate
            y_pred = model.predict(dval)
            weighted_r2 = weighted_r2_score(y_val, y_pred, w_val)

            print(f"Fold {fold + 1} - Weighted R^2: {weighted_r2:.4f}")

            # Store metrics
            fold_weighted_r2_list.append(weighted_r2)

            # Log metrics
            mlflow.log_metric(f"weighted_r2_fold_{fold + 1}", weighted_r2)

            # Extract feature importance
            feature_importance = model.get_score(importance_type='gain')
            sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
            importance_df = pd.DataFrame(sorted_importance, columns=['Feature', 'Importance'])

            # Limit to top 15 features
            top_n = 15
            importance_df = importance_df.head(top_n)

            # Save feature importance to a CSV file
            importance_csv = f'feature_importance_fold_{fold + 1}.csv'
            importance_df.to_csv(importance_csv, index=False)

            # Log the CSV file as an artifact
            mlflow.log_artifact(importance_csv, artifact_path=f'feature_importance/fold_{fold + 1}')

            # Plot feature importance
            plt.figure(figsize=(10, 6))
            importance_df.plot(kind='bar', x='Feature', y='Importance', legend=False)
            plt.title(f'Feature Importance - Fold {fold + 1}')
            plt.xticks(rotation=45, ha='right')  
            plt.tight_layout()

            # Save the plot as an image
            importance_plot = f'feature_importance_fold_{fold + 1}.png'
            plt.savefig(importance_plot)

            # Log the image as an artifact
            mlflow.log_artifact(importance_plot, artifact_path=f'feature_importance/fold_{fold + 1}')

            # Close the plot to free memory
            plt.close()

            # Clean up
            del df_train, df_val, df_train_pd, df_val_pd, X_train, X_val, y_train, y_val, w_train, w_val, dtrain, dval, model
            gc.collect()

        except Exception as e:
            print(f"Error on fold {fold + 1}: {e}")
            mlflow.log_param(f"error_fold_{fold + 1}", str(e))
            continue

    # Calculate average metrics
    avg_weighted_r2 = np.mean(fold_weighted_r2_list)
    std_weighted_r2 = np.std(fold_weighted_r2_list)

    return {
        'avg_weighted_r2': avg_weighted_r2,
        'std_weighted_r2': std_weighted_r2
    }

# Initialize results list
grid_search_results = []

# Begin MLflow run
with mlflow.start_run(run_name="Random Hyperparameter Search with Custom Cross-Validation") as parent_run:
    for idx, params in enumerate(hyperparameter_list):
        print(f"\nEvaluating hyperparameters set {idx + 1}/{n_iter}: {params}")

        # Start a nested MLflow run for this hyperparameter combination
        with mlflow.start_run(run_name=f"Params set {idx + 1}", nested=True):
            # Log hyperparameters
            mlflow.log_params(params)

            # Train and evaluate the model
            metrics = train_evaluate_model(df_full, feature_cols, params, custom_fold_indices)

            # Log average metrics
            mlflow.log_metric("avg_weighted_r2", metrics['avg_weighted_r2'])
            mlflow.log_metric("std_weighted_r2", metrics['std_weighted_r2'])

            # Save results
            result = {
                'params': params,
                'avg_weighted_r2': metrics['avg_weighted_r2'],
                'std_weighted_r2': metrics['std_weighted_r2']
            }
            grid_search_results.append(result)

    # Save grid search results as an artifact

    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, (np.integer, np.int64, np.int32)):
                return int(obj)
            elif isinstance(obj, (np.floating, np.float64, np.float32)):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, (np.bool_)):
                return bool(obj)
            else:
                return super(NumpyEncoder, self).default(obj)

    # At the end of your script
    with open('grid_search_results.json', 'w') as f:
        json.dump(grid_search_results, f, indent=4, cls=NumpyEncoder)
    mlflow.log_artifact('grid_search_results.json')


2024/11/28 16:57:55 INFO mlflow.tracking.fluent: Experiment with name 'Jane Street Forecasting Custom R2' does not exist. Creating a new experiment.



Evaluating hyperparameters set 1/20: {'n_estimators': 90, 'max_depth': 8, 'learning_rate': 0.09338690355288994, 'subsample': 0.943439167964553, 'colsample_bytree': 0.8789472116237456, 'gamma': 0.047088673943824766, 'min_child_weight': 3, 'reg_alpha': 0.761139701990353, 'reg_lambda': 4.037289373746292}

Processing Fold 1
Fold 1 - Weighted R^2: 0.0090

Processing Fold 2
Fold 2 - Weighted R^2: 0.0210

Processing Fold 3
Fold 3 - Weighted R^2: 0.0049

Processing Fold 4
Fold 4 - Weighted R^2: 0.0057

Processing Fold 5
Fold 5 - Weighted R^2: 0.0116

Processing Fold 6
Fold 6 - Weighted R^2: 0.0084

Processing Fold 7
Fold 7 - Weighted R^2: 0.0060


2024/11/28 17:08:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run Params set 1 at: http://localhost:5000/#/experiments/12/runs/6730b587107d4ee0a151613ca53e9dc5.
2024/11/28 17:08:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/12.



Evaluating hyperparameters set 2/20: {'n_estimators': 489, 'max_depth': 6, 'learning_rate': 0.09557332820015775, 'subsample': 0.7483192096930324, 'colsample_bytree': 0.9707059955394407, 'gamma': 0.32193256004033227, 'min_child_weight': 1, 'reg_alpha': 0.82276161327083, 'reg_lambda': 2.49536389472299}

Processing Fold 1
Fold 1 - Weighted R^2: 0.0095

Processing Fold 2
Fold 2 - Weighted R^2: 0.0213

Processing Fold 3
Fold 3 - Weighted R^2: 0.0057

Processing Fold 4
Fold 4 - Weighted R^2: 0.0093

Processing Fold 5
Fold 5 - Weighted R^2: 0.0132

Processing Fold 6
Fold 6 - Weighted R^2: 0.0086

Processing Fold 7
Fold 7 - Weighted R^2: 0.0052


2024/11/28 17:17:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run Params set 2 at: http://localhost:5000/#/experiments/12/runs/d4902e1be5444067a4a1a572bf134fb5.
2024/11/28 17:17:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/12.


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>